# Association between Fertility & Local Municipal Characteristics in Denmark 

### Imports

In [3]:
import geopandas as gpd # for geospatial data handling
import pandas as pd
import contextily as cx # for plotting
import matplotlib.pyplot as plt # for plotting
from pyproj import CRS # for more advanced CRS modifications and transformations

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning) 
warnings.filterwarnings("ignore", category=FutureWarning) 

C:\Users\sutov\AppData\Local\Temp\ipykernel_14516\4267562653.py:1: UserWarning: Shapely 2.0 is installed, but because PyGEOS is also installed, GeoPandas will still use PyGEOS by default for now. To force to use and test Shapely 2.0, you have to set the environment variable USE_PYGEOS=0. You can do this before starting the Python process, or in your code before importing geopandas:

import os
os.environ['USE_PYGEOS'] = '0'
import geopandas

In a future release, GeoPandas will switch to using Shapely by default. If you are using PyGEOS directly (calling PyGEOS functions on geometries from GeoPandas), this will then stop working and you are encouraged to migrate from PyGEOS to Shapely 2.0 (https://shapely.readthedocs.io/en/latest/migration_pygeos.html).
  import geopandas as gpd # for geospatial data handling


### Data

Load the data from DenmarkStatistik and geometries for municipalities of Denmark and then merge them into one geo dataframe

'Data/Educational_attainment_by_highest_education_completed_(total)_2021.xlsx'


In [17]:
data_edu = pd.read_excel('Data/Educational_attainment_by_highest_education_completed_(total)_2021.xlsx')
data_fer = pd.read_excel('Data/fertility_rates_by_municipality_2021.xlsx')
data_bir = pd.read_excel('Data/live_births_by_municipality_2021.xlsx')
data_mar = pd.read_excel('Data/market_value_for_real_estate_average_dkk_2021.xlsx')

In [21]:
# rename the columns to prepare for merge
data_edu.rename(columns = {'Unnamed: 0':'municipality_nm', 2021:'edu_att'}, inplace = True)
data_fer.rename(columns = {'Unnamed: 0':'municipality_nm', 2021:'fer_rate'}, inplace = True)
data_bir.rename(columns = {'Unnamed: 0':'municipality_nm', 2021:'live_birth'}, inplace = True)
data_mar.rename(columns = {'Unnamed: 0':'municipality_nm', 2021:'market_val'}, inplace = True)

In [27]:
# merge to one data frame
data = pd.merge(pd.merge(data_fer, data_bir), pd.merge(data_edu, data_mar))

,municipality_nm,fer_rate,live_birth,edu_att,market_val
0,Copenhagen,1 563.9,10 285,501 212,2 399 736
1,Frederiksberg,1 787.8,1 534,74 788,3 435 510
2,Dragør,2 510.9,146,9 074,4 014 690
3,Tårnby,2 056.7,473,28 727,2 823 345
4,Albertslund,1 608.7,274,18 664,2 931 561
...,...,...,...,...,...
93,Morsø,2 080.7,180,13 103,1 321 820
94,Rebild,2 400.5,380,20 166,1 997 252
95,Thisted,2 063.8,410,28 630,1 422 277
96,Vesthimmerlands,1 824.4,314,24 005,1 495 512


Load the geometries

## Inspecting spatial data

In [ ]:
# just ploting the plygons
cities.plot();

In [ ]:
# quick interactive map
cities.explore()